# Enter data into database

In [4]:
# The re module provides support for regular expressions in Python.
# The sqlite3 module provides a Python interface to the SQLite database.
# The os module provides a way to interact with the operating system, directories, and files.

import re
import sqlite3
import os

# Connect to the database, connect() method takes the name of the database as its argument
conn = sqlite3.connect('student_data.db') #  connects to the database student_data.db, conn = variable stores connection object
c = conn.cursor() # creates a cursor object, cursor object is created by calling the cursor() method on connection object

# Create the tables if they don't already exist
# c.execute() method is used to execute a single SQL statement, take SQL as its argument, returns boolean value
# triple quotes (''') are used to define a multiline string
# multiline string is used to define the SQL statement that creates the table 
c.execute('''CREATE TABLE IF NOT EXISTS tblStudents
             (student_id INTEGER PRIMARY KEY AUTOINCREMENT, fname TEXT, lname TEXT)''')
c.execute('''CREATE TABLE IF NOT EXISTS tblFeedback
             (student_id INTEGER, feedback TEXT, FOREIGN KEY(student_id) REFERENCES tblStudents(student_id))''')
c.execute('''CREATE TABLE IF NOT EXISTS tblScores
             (student_id INTEGER, score INTEGER, FOREIGN KEY(student_id) REFERENCES tblStudents(student_id))''')

# Prompt user for the last name to search for in the directory
lname_search = input("Enter the last name to search for in the directory: ") #input function takes prompt as arg, saved in obj

# Loop through the files in the directory, file = variable to save data
for file in os.listdir(): # iterate over the files in the current directory, os = python module for operating system module
    if file.endswith(lname_search + ".ipynb"): # checks last name with jupyter extension, the endswith() method returns boolean
        # Extract the first and last name from the filename
        # split() method is used to split a string into a list of substrings
        fname_lname = file.split("_")[0].split(".")[0].split("-") # [0] first element of substring, split method of str class
        fname = fname_lname[0] # assign first name of person who created file to variable fname, [0] 1st element of list
        lname = fname_lname[1] # assign last name of person who created file to variable lname, [1] 2nd element of list

        # Prompt user to input feedback and score for this submission
        feedback = input("Enter feedback for " + fname + " " + lname + ": ") # input() method takes string as its argument 
        score = input("Enter score for " + fname + " " + lname + ": ")

        # Insert student data into tblStudents and get the auto-generated student_id
        # c.execute() method is used to execute a SQL, ? placeholders from fname, lname variables 
        c.execute("INSERT INTO tblStudents (fname, lname) VALUES (?, ?)", (fname, lname))
        student_id = c.lastrowid # assign the last row ID that was generated by the database to the variable student_id

        # Insert feedback and score into tblFeedback and tblScores using the student_id
        c.execute("INSERT INTO tblFeedback (student_id, feedback) VALUES (?, ?)", (student_id, feedback))
        c.execute("INSERT INTO tblScores (student_id, score) VALUES (?, ?)", (student_id, score))

# Commit changes and close the database connection
# Conn variable is a connection object used to interact with the database, must commit before closing
conn.commit()
conn.close()


Enter the last name to search for in the directory: johnson
Enter feedback for al johnson: Plagiarism detected
Enter score for al johnson: -1


# Verify data in database via Jupyter

In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('student_data.db')
c = conn.cursor() # creates a cursor object used to execute SQL statements against a database.

# Select all values in tblStudents
c.execute("SELECT * FROM tblStudents")
print("tblStudents:")
for row in c.fetchall(): #  iterates over the results of the c.fetchall() method
    print(row)

# Select all values in tblFeedback
c.execute("SELECT * FROM tblFeedback")
print("tblFeedback:")
for row in c.fetchall():
    print(row)

# Select all values in tblScores
c.execute("SELECT * FROM tblScores")
print("tblScores:")
for row in c.fetchall():
    print(row)

# Close the database connection
conn.close()


tblStudents:
(1, 'joe', 'smith')
(2, 'susan', 'good')
(3, 'rob', 'green')
(4, 'al', 'johnson')
tblFeedback:
(1, 'Good work')
(2, 'Incorrect format')
(3, 'Plagiarism detected')
(4, 'Plagiarism detected')
tblScores:
(1, 1)
(2, 0)
(3, -1)
(4, -1)


# Delete student scores

This Python code will prompt the user to enter a student_id to search for in the tblStudents table for the SQLite database, and then the code will delete the student score associated with the student ID found and the associated rows in both the tblFeedback and tblScores tables. 

In [19]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('student_data.db')
c = conn.cursor()

# Prompt the user to enter the student_id to search for
student_id = input("Enter a student_id to delete scores for: ")

# Check if the student_id exists in the tblStudents table
c.execute("SELECT * FROM tblStudents WHERE student_id = ?", (student_id,))
if not c.fetchone():
    print("Invalid student_id.")
    exit()

# Delete the student's scores
c.execute("DELETE FROM tblStudents WHERE student_id = ?", (student_id,))
c.execute("DELETE FROM tblScores WHERE student_id = ?", (student_id,))
c.execute("DELETE FROM tblFeedback WHERE student_id = ?", (student_id,))

# Commit the changes to the database
conn.commit()

# Select all values in tblStudents
c.execute("SELECT * FROM tblStudents")
print("tblStudents:")
for row in c.fetchall(): #  iterates over the results of the c.fetchall() method
    print(row)

# Select all values in tblFeedback
c.execute("SELECT * FROM tblFeedback")
print("tblFeedback:")
for row in c.fetchall():
    print(row)

# Select all values in tblScores
c.execute("SELECT * FROM tblScores")
print("tblScores:")
for row in c.fetchall():
    print(row)

# Close the database connection
conn.close()


Enter a student_id to delete scores for: 2
tblStudents:
(1, 'joe', 'smith')
(3, 'rob', 'green')
(4, 'al', 'johnson')
tblFeedback:
(1, 'Good work')
(3, 'Plagiarism detected')
(4, 'Plagiarism detected')
tblScores:
(1, 1)
(3, -1)
(4, -1)
